In [1]:
import yaml

# Set `PATH` to include the directory containing TFX CLI and skaffold.
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [26]:
!cmd()

/bin/bash: -c: line 1: syntax error: unexpected end of file


In [2]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.25.0
KFP version: 1.0.4


In [6]:
!ls

__pycache__	model.py	  tfx-pipeline-components.ipynb
artifact-store	preprocessing.py  tfx-pipeline.ipynb
features.py	serving_model	  tutorials
mlops-on-gcp	src


In [7]:
!gsutil ls

gs://dataprep-staging-8d399cfa-977e-4458-ad47-fef87f7b3896/
gs://gcp_churn_bucket/
gs://gcp_taxirides_bucket/
gs://imperial-legacy-380605-kubeflowpipelines-default/
gs://tfx_pipeline_bucket/


In [8]:
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://imperial-legacy-380605-kubeflowpipelines-default'
ENDPOINT = 'https://1cfc597c9414e46e-dot-us-central1.pipelines.googleusercontent.com'
CUSTOM_SERVICE_ACCOUNT = 'tfx-tuner-caip-service-account@imperial-legacy-380605.iam.gserviceaccount.com'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [10]:
%env GCP_REGION={GCP_REGION}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env CUSTOM_SERVICE_ACCOUNT={CUSTOM_SERVICE_ACCOUNT}
%env PROJECT_ID={PROJECT_ID}

env: GCP_REGION=us-central1
env: ARTIFACT_STORE_URI=gs://imperial-legacy-380605-kubeflowpipelines-default
env: CUSTOM_SERVICE_ACCOUNT=tfx-tuner-caip-service-account@imperial-legacy-380605.iam.gserviceaccount.com
env: PROJECT_ID=imperial-legacy-380605


In [11]:
PIPELINE_NAME = 'tfx_churn_pipeline'
MODEL_NAME = 'tfx_churn_classifier'
DATA_ROOT_URI = 'gs://tfx_pipeline_bucket'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)
RUNTIME_VERSION = '2.3'
PYTHON_VERSION = '3.7'
USE_KFP_SA=False
ENABLE_TUNING=False

In [12]:
%env PIPELINE_NAME={PIPELINE_NAME}
%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env KUBEFLOW_TFX_IMAGE={CUSTOM_TFX_IMAGE}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERIONS={PYTHON_VERSION}
%env USE_KFP_SA={USE_KFP_SA}
%env ENABLE_TUNING={ENABLE_TUNING}

env: PIPELINE_NAME=tfx_churn_pipeline
env: MODEL_NAME=tfx_churn_classifier
env: DATA_ROOT_URI=gs://tfx_pipeline_bucket
env: KUBEFLOW_TFX_IMAGE=gcr.io/imperial-legacy-380605/tfx_churn_pipeline
env: RUNTIME_VERSION=2.3
env: PYTHON_VERIONS=3.7
env: USE_KFP_SA=False
env: ENABLE_TUNING=False


In [13]:
!tfx pipeline compile --engine kubeflow --pipeline_path runner.py

2023-04-20 11:28:46.276502: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 11:28:46.276551: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Compiling pipeline
2023-04-20 11:28:52.827779: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 11:28:52.827828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Pipeline 

In [14]:
!tfx pipeline create  \
--pipeline_path=runner.py \
--endpoint={ENDPOINT} \
--build_target_image={CUSTOM_TFX_IMAGE}

2023-04-20 11:32:30.279868: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 11:32:30.279919: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
No local setup.py, copying the directory and configuring the PYTHONPATH.
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/imperial-legacy-380605/tfx_churn_pipeline -> gcr.io/imperial-legacy-380605/tfx_churn_pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/imperial-legacy-380605/tfx_churn_pipeline: Not found. Building
[Skaffold] Starting build...
[Skaffol

In [22]:
# !tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}

2023-04-20 12:28:12.705646: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 12:28:12.705707: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/imperial-legacy-380605/tfx_churn_pipeline -> gcr.io/imperial-legacy-380605/tfx_churn_pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/imperial-legacy-380605/tfx_churn_pipeline: Not found. Building
[Skaffold] Starting build...
[Skaffold] Building [gcr.io/imperial-legacy-380605/tfx_churn_pipeline]...
[Skaffo

In [23]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

2023-04-20 12:33:58.398801: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 12:33:58.398858: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating a run for pipeline: tfx_churn_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: tfx_churn_pipeline
+--------------------+--------------------------------------+----------+---------------------------+------------------------------------------------------------------------------------------------------------------------------+
| pipeline_name      | run_id                               | status   | created_at                | link  

In [16]:
!tfx run list --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

2023-04-20 11:44:11.318023: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 11:44:11.318080: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Listing all runs of pipeline: tfx_churn_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
+--------------------+--------------------------------------+----------+---------------------------+------------------------------------------------------------------------------------------------------------------------------+
| pipeline_name      | run_id                               | status   | created_at                | link                                              